In [22]:
from collections import Counter, defaultdict
import json
import pdb

import numpy as np
import pandas as pd

# Redo filtering

In [2]:
# with open("/data/wei/hpa-webdataset-all-composite/HPACombineDatasetInfo.pickle", "rb") as in_file:
#     data_info = pickle.load(in_file)
# len(data_info), data_info[0]
meta_data = pd.read_csv("/scratch/users/xikunz2/stable-diffusion/hpa-webdataset-all-composite/v23/IF-image.csv")
meta_data

/tmp/ipykernel_109378/2619399869.py:4: DtypeWarning: Columns (7,8,14,16,18,19,33) have mixed types. Specify dtype option on import or set low_memory=False.
  meta_data = pd.read_csv("/scratch/users/xikunz2/stable-diffusion/hpa-webdataset-all-composite/v23/IF-image.csv")


,filename,jpg_prefix,if_plate_id,position,sample,status,Image status name,locations,staining characteristics,unspecific,...,z_pos,Experiment state,Ab state,Max tpm,Finished in genes,Protocol,Gene reliability (in release),Gene reliability (lims),Cell count,well_location_predictions_all
0,/archive/1/1_A1_1_,https://lims.proteinatlas.org/images/992/1_A1_1_,1,A1,1,35,Annotated / Proteinatlas,Golgi apparatus,NaN,0.0,...,NaN,IF_FINISHED,IF_FINISHED,16.13,ENSG00000066455,PFA,Supported,Supported,7.0,NaN
1,/archive/1/1_A1_2_,https://lims.proteinatlas.org/images/992/1_A1_2_,1,A1,2,35,Annotated / Proteinatlas,Golgi apparatus,NaN,0.0,...,NaN,IF_FINISHED,IF_FINISHED,16.13,ENSG00000066455,PFA,Supported,Supported,6.0,NaN
2,/archive/1/1_A10_1_,https://lims.proteinatlas.org/images/1202/1_A1...,1,A10,1,35,Annotated / Proteinatlas,Cytosol,NaN,0.0,...,NaN,IF_FINISHED,IF_FAILED,30.50,NaN,PFA,NaN,NaN,5.0,NaN
3,/archive/1/1_A10_2_,https://lims.proteinatlas.org/images/1202/1_A1...,1,A10,2,35,Annotated / Proteinatlas,Cytosol,NaN,0.0,...,NaN,IF_FINISHED,IF_FAILED,30.50,NaN,PFA,NaN,NaN,4.0,NaN
4,/archive/1/1_A11_1_,https://lims.proteinatlas.org/images/4907/1_A1...,1,A11,1,35,Annotated / Proteinatlas,NaN,NaN,1.0,...,NaN,IF_FAILED_REVIEW,IF_FAILED,0.07,NaN,PFA,NaN,NaN,3.0,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1086885,/archive/2184/2184_H8_20_,https://lims.proteinatlas.org/images/5751/2184...,2184,H8,20,20,Pending annotation,NaN,NaN,NaN,...,0.003399,IF_IN_SCANNING,IF_FURTHER_EVAL,NaN,NaN,PFA,Approved,NaN,NaN,NaN
1086886,/archive/2184/2184_H8_21_,https://lims.proteinatlas.org/images/5751/2184...,2184,H8,21,20,Pending annotation,NaN,NaN,NaN,...,0.003398,IF_IN_SCANNING,IF_FURTHER_EVAL,NaN,NaN,PFA,Approved,NaN,NaN,NaN
1086887,/archive/2184/2184_H8_22_,https://lims.proteinatlas.org/images/5751/2184...,2184,H8,22,20,Pending annotation,NaN,NaN,NaN,...,0.003397,IF_IN_SCANNING,IF_FURTHER_EVAL,NaN,NaN,PFA,Approved,NaN,NaN,NaN
1086888,/archive/2184/2184_H8_23_,https://lims.proteinatlas.org/images/5751/2184...,2184,H8,23,20,Pending annotation,NaN,NaN,NaN,...,0.003396,IF_IN_SCANNING,IF_FURTHER_EVAL,NaN,NaN,PFA,Approved,NaN,NaN,NaN


In [3]:
data_info = [row for _, row in meta_data.iterrows()]
data_info[0]

filename                                                        /archive/1/1_A1_1_
jpg_prefix                        https://lims.proteinatlas.org/images/992/1_A1_1_
if_plate_id                                                                      1
position                                                                        A1
sample                                                                           1
status                                                                          35
Image status name                                         Annotated / Proteinatlas
locations                                                          Golgi apparatus
staining characteristics                                                       NaN
unspecific                                                                     0.0
antibody                                                                 HPA000992
ensembl_ids                                                        ENSG00000066455
gene

In [4]:
ab_states = [x["Ab state"] for x in data_info]
Counter(ab_states).most_common()

[('IF_FINISHED', 404567),
 ('IF_FAILED', 396015),
 ('IF_FURTHER_EVAL', 212113),
 ('IF_IN_PROGRESS', 66486),
 ('IF_PENDING_DESTINY', 7679),
 ('IF_PENDING', 30)]

In [6]:
filter_func = lambda x: int( x['status']) == 35 and x['Ab state'] == 'IF_FINISHED' and str(x['locations']) != "nan"
filter_func2 = lambda x: int( x['status']) == 35 and x['Ab state'] == 'IF_FINISHED'
filter_func3 = lambda x: int( x['status']) == 35 and x['Ab state'] == 'IF_FINISHED' and str(x["gene_names"]) != "nan"
# filter_func4 = lambda x: int( x['status']) == 35 and x['Ab state'] == 'IF_FINISHED' and str(x["gene_names"]) != "nan" and x["sequences"]
flt_data_info = list(filter(filter_func, data_info))
flt2_data_info = list(filter(filter_func2, data_info))
flt3_data_info = list(filter(filter_func3, data_info))
# flt4_data_info = list(filter(filter_func4, data_info))
len(flt_data_info), len(flt2_data_info), len(flt3_data_info)

(92339, 108511, 106875)

In [7]:
flt3_idcs = []
for i, x in enumerate(data_info):
    if filter_func3(x):
        flt3_idcs.append(i)
len(flt3_idcs)

106875

In [8]:
for x in flt3_data_info:
    prot = x["gene_names"]
    if "," in prot:
        print(prot)

IGBP1,IGBP1P2
IGBP1,IGBP1P2
PBX1,PBX3
PBX1,PBX3
KRT14,KRT16,KRT17
KRT14,KRT16,KRT17
IGBP1,IGBP1P2
IGBP1,IGBP1P2
PBX1,PBX3
PBX1,PBX3
IGBP1,IGBP1P2
IGBP1,IGBP1P2
PBX1,PBX3
PBX1,PBX3
KRT14,KRT17
KRT14,KRT17
KRT14,KRT16,KRT17
KRT14,KRT16,KRT17
SEPTIN11,SEPTIN6,SEPTIN8
SEPTIN11,SEPTIN6,SEPTIN8
SEPTIN11,SEPTIN6,SEPTIN8
SEPTIN11,SEPTIN6,SEPTIN8
SEPTIN11,SEPTIN6,SEPTIN8
SEPTIN11,SEPTIN6,SEPTIN8
ISY1,ISY1-RAB43
ISY1,ISY1-RAB43
NME1,NME1-NME2
NME1,NME1-NME2
ISY1,ISY1-RAB43
ISY1,ISY1-RAB43
NME1,NME1-NME2
NME1,NME1-NME2
ISY1,ISY1-RAB43
ISY1,ISY1-RAB43
NME1,NME1-NME2
NME1,NME1-NME2
BCL2L2-PABPN1,PABPN1
BCL2L2-PABPN1,PABPN1
SYNJ2BP,SYNJ2BP-COX16
SYNJ2BP,SYNJ2BP-COX16
SLC2A14,SLC2A3
SLC2A14,SLC2A3
BCL2L2-PABPN1,PABPN1
BCL2L2-PABPN1,PABPN1
SYNJ2BP,SYNJ2BP-COX16
SYNJ2BP,SYNJ2BP-COX16
SLC2A14,SLC2A3
SLC2A14,SLC2A3
DDX3X,DDX3Y
DDX3X,DDX3Y
ECH1,ENSG00000268083
ECH1,ENSG00000268083
DDX3X,DDX3Y
DDX3X,DDX3Y
DDX3X,DDX3Y
DDX3X,DDX3Y
DDX3X,DDX3Y
DDX3X,DDX3Y
BCL2L2-PABPN1,PABPN1
BCL2L2-PABPN1,PABPN1
SYNJ2BP,SYNJ

# Create a protein-cell-line-pair imputation data split

In [31]:
comb2img_idcs, prot2img_idcs = defaultdict(list), defaultdict(list)
for i in flt3_idcs:
    x = data_info[i]
    prot, cl = x["gene_names"], x['atlas_name']
    assert type(prot) != float
    comb2img_idcs[(prot, cl)].append(i)
    prot2img_idcs[prot].append(i)
len(comb2img_idcs), len(prot2img_idcs)

(44873, 12505)

In [18]:
Counter(map(len, comb2img_idcs.values()))

Counter({2: 37367,
         8: 136,
         4: 6416,
         6: 751,
         10: 37,
         5: 11,
         11: 1,
         1: 87,
         14: 3,
         3: 50,
         12: 14})

In [21]:
np.random.seed(42)

train_idcs, valid_idcs, test_idcs = [], [], []
for imgidcs in comb2img_idcs.values():
    n_imgs = len(imgidcs)
    if n_imgs < 4:
        train_idcs.extend(imgidcs)
    else:
        np.random.shuffle(imgidcs)
        train_idcs.extend(imgidcs[:int(0.6 * n_imgs)])
        valid_idcs.extend(imgidcs[int(0.6 * n_imgs):int(0.8 * n_imgs)])
        test_idcs.extend(imgidcs[int(0.8 * n_imgs):])
len(train_idcs), len(valid_idcs), len(test_idcs)

(90983, 7563, 8329)

In [28]:
# meta_data.loc[train_idcs, "imputation_split"] = "train"
# meta_data.loc[valid_idcs, "imputation_split"] = "validation"
# meta_data.loc[test_idcs, "imputation_split"] = "test"
# meta_data

,filename,jpg_prefix,if_plate_id,position,sample,status,Image status name,locations,staining characteristics,unspecific,...,Experiment state,Ab state,Max tpm,Finished in genes,Protocol,Gene reliability (in release),Gene reliability (lims),Cell count,well_location_predictions_all,imputation_split
0,/archive/1/1_A1_1_,https://lims.proteinatlas.org/images/992/1_A1_1_,1,A1,1,35,Annotated / Proteinatlas,Golgi apparatus,NaN,0.0,...,IF_FINISHED,IF_FINISHED,16.13,ENSG00000066455,PFA,Supported,Supported,7.0,NaN,train
1,/archive/1/1_A1_2_,https://lims.proteinatlas.org/images/992/1_A1_2_,1,A1,2,35,Annotated / Proteinatlas,Golgi apparatus,NaN,0.0,...,IF_FINISHED,IF_FINISHED,16.13,ENSG00000066455,PFA,Supported,Supported,6.0,NaN,train
2,/archive/1/1_A10_1_,https://lims.proteinatlas.org/images/1202/1_A1...,1,A10,1,35,Annotated / Proteinatlas,Cytosol,NaN,0.0,...,IF_FINISHED,IF_FAILED,30.50,NaN,PFA,NaN,NaN,5.0,NaN,NaN
3,/archive/1/1_A10_2_,https://lims.proteinatlas.org/images/1202/1_A1...,1,A10,2,35,Annotated / Proteinatlas,Cytosol,NaN,0.0,...,IF_FINISHED,IF_FAILED,30.50,NaN,PFA,NaN,NaN,4.0,NaN,NaN
4,/archive/1/1_A11_1_,https://lims.proteinatlas.org/images/4907/1_A1...,1,A11,1,35,Annotated / Proteinatlas,NaN,NaN,1.0,...,IF_FAILED_REVIEW,IF_FAILED,0.07,NaN,PFA,NaN,NaN,3.0,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1086885,/archive/2184/2184_H8_20_,https://lims.proteinatlas.org/images/5751/2184...,2184,H8,20,20,Pending annotation,NaN,NaN,NaN,...,IF_IN_SCANNING,IF_FURTHER_EVAL,NaN,NaN,PFA,Approved,NaN,NaN,NaN,NaN
1086886,/archive/2184/2184_H8_21_,https://lims.proteinatlas.org/images/5751/2184...,2184,H8,21,20,Pending annotation,NaN,NaN,NaN,...,IF_IN_SCANNING,IF_FURTHER_EVAL,NaN,NaN,PFA,Approved,NaN,NaN,NaN,NaN
1086887,/archive/2184/2184_H8_22_,https://lims.proteinatlas.org/images/5751/2184...,2184,H8,22,20,Pending annotation,NaN,NaN,NaN,...,IF_IN_SCANNING,IF_FURTHER_EVAL,NaN,NaN,PFA,Approved,NaN,NaN,NaN,NaN
1086888,/archive/2184/2184_H8_23_,https://lims.proteinatlas.org/images/5751/2184...,2184,H8,23,20,Pending annotation,NaN,NaN,NaN,...,IF_IN_SCANNING,IF_FURTHER_EVAL,NaN,NaN,PFA,Approved,NaN,NaN,NaN,NaN


In [25]:
max(train_idcs), max(valid_idcs), max(test_idcs)

(1077598, 1045119, 1045546)

# Create a protein-cell-line-pair denovo data split

In [26]:
all_combs = list(comb2img_idcs.keys())
np.random.shuffle(all_combs)
num_combs = len(all_combs)
train_combs = all_combs[:int(0.6 * num_combs)]
valid_combs = all_combs[int(0.6 * num_combs):int(0.8 * num_combs)]
test_combs = all_combs[int(0.8 * num_combs):]
train_prots = set([prot for (prot, cl) in train_combs])
train_cls = set([cl for (prot, cl) in train_combs])
flt_valid_combs = []
for (prot, cl) in valid_combs:
    if prot not in train_prots or cl not in train_cls:
        train_combs.append((prot, cl))
        train_prots.add(prot)
        train_cls.add(cl)
    else:
        flt_valid_combs.append((prot, cl))
len(all_combs), len(train_combs), len(flt_valid_combs)

(44873, 27505, 8393)

In [27]:
flt_test_combs = []
for (prot, cl) in test_combs:
    if prot not in train_prots or cl not in train_cls:
        train_combs.append((prot, cl))
        train_prots.add(prot)
        train_cls.add(cl)
    else:
        flt_test_combs.append((prot, cl))
len(all_combs), len(train_combs), len(flt_test_combs)

(44873, 27582, 8898)

In [29]:
train_idcs, valid_idcs, test_idcs = [], [], []
for comb in train_combs:
    train_idcs.extend(comb2img_idcs[comb])
for comb in valid_combs:
    valid_idcs.extend(comb2img_idcs[comb])
for comb in test_combs:
    test_idcs.extend(comb2img_idcs[comb])
len(train_idcs), len(valid_idcs), len(test_idcs)

(65515, 21408, 21466)

In [30]:
# meta_data.loc[train_idcs, "protcl_denovo_split"] = "train"
# meta_data.loc[valid_idcs, "protcl_denovo_split"] = "validation"
# meta_data.loc[test_idcs, "protcl_denovo_split"] = "test"
# meta_data

,filename,jpg_prefix,if_plate_id,position,sample,status,Image status name,locations,staining characteristics,unspecific,...,Ab state,Max tpm,Finished in genes,Protocol,Gene reliability (in release),Gene reliability (lims),Cell count,well_location_predictions_all,imputation_split,protcl_denovo_split
0,/archive/1/1_A1_1_,https://lims.proteinatlas.org/images/992/1_A1_1_,1,A1,1,35,Annotated / Proteinatlas,Golgi apparatus,NaN,0.0,...,IF_FINISHED,16.13,ENSG00000066455,PFA,Supported,Supported,7.0,NaN,train,validation
1,/archive/1/1_A1_2_,https://lims.proteinatlas.org/images/992/1_A1_2_,1,A1,2,35,Annotated / Proteinatlas,Golgi apparatus,NaN,0.0,...,IF_FINISHED,16.13,ENSG00000066455,PFA,Supported,Supported,6.0,NaN,train,validation
2,/archive/1/1_A10_1_,https://lims.proteinatlas.org/images/1202/1_A1...,1,A10,1,35,Annotated / Proteinatlas,Cytosol,NaN,0.0,...,IF_FAILED,30.50,NaN,PFA,NaN,NaN,5.0,NaN,NaN,NaN
3,/archive/1/1_A10_2_,https://lims.proteinatlas.org/images/1202/1_A1...,1,A10,2,35,Annotated / Proteinatlas,Cytosol,NaN,0.0,...,IF_FAILED,30.50,NaN,PFA,NaN,NaN,4.0,NaN,NaN,NaN
4,/archive/1/1_A11_1_,https://lims.proteinatlas.org/images/4907/1_A1...,1,A11,1,35,Annotated / Proteinatlas,NaN,NaN,1.0,...,IF_FAILED,0.07,NaN,PFA,NaN,NaN,3.0,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1086885,/archive/2184/2184_H8_20_,https://lims.proteinatlas.org/images/5751/2184...,2184,H8,20,20,Pending annotation,NaN,NaN,NaN,...,IF_FURTHER_EVAL,NaN,NaN,PFA,Approved,NaN,NaN,NaN,NaN,NaN
1086886,/archive/2184/2184_H8_21_,https://lims.proteinatlas.org/images/5751/2184...,2184,H8,21,20,Pending annotation,NaN,NaN,NaN,...,IF_FURTHER_EVAL,NaN,NaN,PFA,Approved,NaN,NaN,NaN,NaN,NaN
1086887,/archive/2184/2184_H8_22_,https://lims.proteinatlas.org/images/5751/2184...,2184,H8,22,20,Pending annotation,NaN,NaN,NaN,...,IF_FURTHER_EVAL,NaN,NaN,PFA,Approved,NaN,NaN,NaN,NaN,NaN
1086888,/archive/2184/2184_H8_23_,https://lims.proteinatlas.org/images/5751/2184...,2184,H8,23,20,Pending annotation,NaN,NaN,NaN,...,IF_FURTHER_EVAL,NaN,NaN,PFA,Approved,NaN,NaN,NaN,NaN,NaN


# Create a protein denovo data split

In [33]:
all_prots = list(prot2img_idcs.keys())
np.random.shuffle(all_prots)
num_prots = len(all_prots)
train_prots = all_prots[:int(0.6 * num_prots)]
valid_prots = all_prots[int(0.6 * num_prots):int(0.8 * num_prots)]
test_prots = all_prots[int(0.8 * num_prots):]
len(all_prots), len(train_prots), len(valid_prots), len(test_prots)

(12505, 7503, 2501, 2501)

In [34]:
train_idcs, valid_idcs, test_idcs = [], [], []
for prot in train_prots:
    train_idcs.extend(prot2img_idcs[prot])
for prot in valid_prots:
    valid_idcs.extend(prot2img_idcs[prot])
for prot in test_prots:
    test_idcs.extend(prot2img_idcs[prot])
len(train_idcs), len(valid_idcs), len(test_idcs)

(63920, 21784, 21171)

In [35]:
# meta_data.loc[train_idcs, "prot_denovo_split"] = "train"
# meta_data.loc[valid_idcs, "prot_denovo_split"] = "validation"
# meta_data.loc[test_idcs, "prot_denovo_split"] = "test"
# meta_data

,filename,jpg_prefix,if_plate_id,position,sample,status,Image status name,locations,staining characteristics,unspecific,...,Max tpm,Finished in genes,Protocol,Gene reliability (in release),Gene reliability (lims),Cell count,well_location_predictions_all,imputation_split,protcl_denovo_split,prot_denovo_split
0,/archive/1/1_A1_1_,https://lims.proteinatlas.org/images/992/1_A1_1_,1,A1,1,35,Annotated / Proteinatlas,Golgi apparatus,NaN,0.0,...,16.13,ENSG00000066455,PFA,Supported,Supported,7.0,NaN,train,validation,test
1,/archive/1/1_A1_2_,https://lims.proteinatlas.org/images/992/1_A1_2_,1,A1,2,35,Annotated / Proteinatlas,Golgi apparatus,NaN,0.0,...,16.13,ENSG00000066455,PFA,Supported,Supported,6.0,NaN,train,validation,test
2,/archive/1/1_A10_1_,https://lims.proteinatlas.org/images/1202/1_A1...,1,A10,1,35,Annotated / Proteinatlas,Cytosol,NaN,0.0,...,30.50,NaN,PFA,NaN,NaN,5.0,NaN,NaN,NaN,NaN
3,/archive/1/1_A10_2_,https://lims.proteinatlas.org/images/1202/1_A1...,1,A10,2,35,Annotated / Proteinatlas,Cytosol,NaN,0.0,...,30.50,NaN,PFA,NaN,NaN,4.0,NaN,NaN,NaN,NaN
4,/archive/1/1_A11_1_,https://lims.proteinatlas.org/images/4907/1_A1...,1,A11,1,35,Annotated / Proteinatlas,NaN,NaN,1.0,...,0.07,NaN,PFA,NaN,NaN,3.0,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1086885,/archive/2184/2184_H8_20_,https://lims.proteinatlas.org/images/5751/2184...,2184,H8,20,20,Pending annotation,NaN,NaN,NaN,...,NaN,NaN,PFA,Approved,NaN,NaN,NaN,NaN,NaN,NaN
1086886,/archive/2184/2184_H8_21_,https://lims.proteinatlas.org/images/5751/2184...,2184,H8,21,20,Pending annotation,NaN,NaN,NaN,...,NaN,NaN,PFA,Approved,NaN,NaN,NaN,NaN,NaN,NaN
1086887,/archive/2184/2184_H8_22_,https://lims.proteinatlas.org/images/5751/2184...,2184,H8,22,20,Pending annotation,NaN,NaN,NaN,...,NaN,NaN,PFA,Approved,NaN,NaN,NaN,NaN,NaN,NaN
1086888,/archive/2184/2184_H8_23_,https://lims.proteinatlas.org/images/5751/2184...,2184,H8,23,20,Pending annotation,NaN,NaN,NaN,...,NaN,NaN,PFA,Approved,NaN,NaN,NaN,NaN,NaN,NaN


In [36]:
# meta_data.to_csv("/scratch/users/xikunz2/stable-diffusion/hpa-webdataset-all-composite/v23/IF-image-w-splits.csv")